# Analyzing SOFC data (part 5)

##5) Finalizing the model
So far, I am only confident that two features should be in the model: average electron affinity and average d-count for the B-cation. I have ruled out the parent features and now look only at the average features. I have no yet completely ruled out tolerance factor and critical radius, but I don't believe they will add much.

So what subset of features do I choose for my model?

In this notebook, I will determine the best subset of features by only allowing features that contribute at least 0.05 to the average model score after electron affinity and d-electron count have been added to it. At the time of this writing, sklearn has not implemented a best subset feature selection methodology, so I will have to build one myself. If I had many features, I would write a function to do this recursively, but because I only have a few, I believe it would be more efficient and more instructive to do it step-by-step.

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import pandas as pd
import time
%matplotlib inline
from sklearn import cross_validation
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFE

from sofc_func import *

In [17]:
# Import using pandas
df = pd.read_csv("data.csv")

# Clean out rows where there is no parent A or parent B

data = df[pd.notnull(df['A_par']) & pd.notnull(df['B_par']) & pd.notnull(df['d_star']) & pd.notnull(df['k_star']) 
             & pd.notnull(df['e affinity(B)']) & pd.notnull(df['d-electron count (B)'])]

pd.options.mode.chained_assignment = None

data = features(df)

data['dk_star'] = pd.Series(dk_star(data,1000), index=df.index)

In [18]:
f = df[pd.notnull(data['EA_A']) & pd.notnull(data['EA_B']) & pd.notnull(data['r_A'])
          & pd.notnull(data['r_B']) & pd.notnull(data['d_count_B']) & pd.notnull(data['avg_EA_A'])
          & pd.notnull(data['avg_EA_B']) & pd.notnull(data['avg_r_A']) & pd.notnull(data['avg_r_B'])
          & pd.notnull(data['avg_d_count_B']) & pd.notnull(data['dk_star'])]

X = f[ ['avg_EA_B', 'avg_d_count_B'] ]
y = f['dk_star']
features = f[ ['avg_EA_A', 'avg_r_A','avg_r_B', 'tol_factor', 'r_critical' ]]

In [32]:
def add_feature(X, Y, features, threshold, iterations):
    start = time.clock()
    print 'Determining if a feature can be added. This may take a few minutes.'
    print np.array(X).shape, np.array(Y).shape
    k = optimize_k(iterations, len(X), X, Y, True)
    #o = optimize_order(iterations, len(X), k, X, Y, True)
    #_score, _std = get_scores(iterations, k, o, len(X), X, Y, True)

    '''
    for f in features:
        test_x = X
        test_x[f] = features[a]
        k = optimize_k(iterations, len(test_x), test_x, Y, True)
        o = optimize_order(iterations,len(test_x), k, test_x, Y, True)
        new_score, std = get_scores(iterations, k, o, len(test_x), test_x, Y, True)
        if new_score - _score > threshold:
            print 'Added ' + f + ' to the model, which boosts the score from ' + _score + ' to ' + new_score
            print 'Executed in ' + (time.clock()-start) + 's'
            return test_x
    '''
    print 'Could not add any features to the model.'
    print 'Executed in ' , (time.clock()-start) , 's'
    return X

In [33]:
new = add_feature(X, np.array([y]).T, features, 0.04, 100)

Determining if a feature can be added. This may take a few minutes.
(94, 2) (94, 1)


IndexError: indices are out-of-bounds